In [1]:
#Bone Fracture Detection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/MURA-v1.1.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Completed")

Completed


In [3]:
import os
import numpy as np
import cv2
import random
import pandas as pd
from tqdm import tqdm
import keras.backend as K
import matplotlib.pyplot as plt
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras.preprocessing.image import * #image
from keras import regularizers
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.applications.densenet import DenseNet169, DenseNet121, preprocess_input
from time import time




import datetime

In [4]:
!pip install focal-loss
!pip install keras-radam

  Created wheel for keras-radam: filename=keras_radam-0.15.0-py3-none-any.whl size=14686 sha256=3825fa784b1c973baa88d8e89fd2499f7efbbd7751700198382eca7d367d2ece
  Stored in directory: /root/.cache/pip/wheels/23/6a/5f/d674f3b7b4d504b03148abd675e3703ba00c31763c04a2fc20
Successfully built keras-radam


In [5]:

from focal_loss import BinaryFocalLoss
from keras_radam import RAdam


In [6]:
def listdir_nohidden(path):
    '''
    Utility function to find the list of files in a directory excluding the hidden files.
    Args:
        path: contains the path of the directory containing the images

    '''
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
# this will basically ingnore hidden files

In [7]:
def create_images_metadata_csv(category,study_types):
    '''
    This function creates a csv file containing the path of images, label.
    Args:
        category: train or valid, depending on which csv is needed
        study_types: the type of the body part in MURA Dataset, eg:  XR_SHOULDER
    '''
    image_data = {}
    study_label = {'positive': 1, 'negative': 0}
    #study_types = ['XR_ELBOW','XR_FINGER','XR_FOREARM','XR_HAND','XR_HUMERUS','XR_SHOULDER','XR_WRIST']
    #study_types = ['XR_ELBOW']
    i = 0
    image_data[category] = pd.DataFrame(columns=['Path','Count', 'Label'])
    for study_type in study_types: # Iterate throught every study types
        DATA_DIR = 'MURA-v1.1/%s/%s/' % (category, study_type)
        patients = list(os.walk(DATA_DIR))[0][1]  # list of patient folder names
        for patient in tqdm(patients):  # for each patient folder
            for study in os.listdir(DATA_DIR + patient):  # for each study in that patient folder
                if(study != '.DS_Store'):
                    label = study_label[study.split('_')[1]]  # get label 0 or 1
                    path = DATA_DIR + patient + '/' + study + '/'  # path to this study
                    for j in range(len(list(listdir_nohidden(path)))):
                        image_path = path + 'image%s.png' % (j + 1)
                        image_data[category].loc[i] = [image_path,1, label]  # add new row
                        i += 1
    image_data[category].to_csv(category+"_image_data.csv",index = None, header=False)

In [8]:
def getImageLabels(dataframe):
  '''
  This function is to get the labels of the images from the dataframe
  Args:
      dataframe: pandas dataframe containing the labels of the images

  '''
  labels = []
  for i, data in tqdm(dataframe.iterrows()):

      labels.append(data['Label'])

  labels = np.asarray(labels)
  return labels

In [9]:
def getImage(dataframe, size):
    '''
    Function to process the images
    Args:
        dataframe: contains the path to the images in the directory
        size: contains the value to which the shape of the image will resized
    '''
    Images = []
    for i, data in tqdm(dataframe.iterrows()):
      try:
        image = cv2.imread(data['Path'])#cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(size,size))
        #image = randome_rotation_flip(image,size)
        Images.append(image)

      except Exception as e:
        print(str(e))

    Images = np.asarray(Images).astype('float32')

    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    
  
    return Images

In [10]:
def build_model():
    base_model = DenseNet169(input_shape=(None, None,3),
                             weights='imagenet',
                             include_top=False,
                             pooling='avg')


    x = base_model.output

    predictions = Dense(n_classes,activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

## **DATA PROCESSING**

In [11]:
img_width, img_height = 224, 224
#Keras ImageDataGenerator to load, transform the images of the dataset

IMG_DATA_DIR = 'MURA-v1.1/'
train_data_dir = IMG_DATA_DIR + 'train/XR_HUMERUS'
valid_data_dir = IMG_DATA_DIR + 'valid/XR_HUMERUS'

train_datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True

)
study_types = ['XR_HUMERUS']

#TRAIN DATA
create_images_metadata_csv('train',study_types)
#VALID DATA
create_images_metadata_csv('valid',study_types)

100%|██████████| 132/132 [00:00<00:00, 161.41it/s]


In [12]:
valid_image_df = pd.read_csv('valid_image_data.csv', names=['Path','Count', 'Label'])
train_image_df = pd.read_csv('train_image_data.csv', names=['Path', 'Count','Label'])

dd={}

dd['train'] = train_image_df
dd['valid'] = valid_image_df

In [13]:
train_image_labels = getImageLabels(train_image_df)
train_images = getImage(train_image_df, size = 224)

1272it [00:00, 10696.22it/s]
1272it [00:05, 233.03it/s]


In [14]:
train_images.shape
train_datagen.fit(train_images,augment=True)
train_generator = train_datagen.flow(
    x=train_images,
    y=train_image_labels,
    batch_size = 32
)

In [15]:
valid_image_labels = getImageLabels(valid_image_df)

288it [00:00, 12964.56it/s]


In [16]:
valid_images = getImage(valid_image_df, size = 224)

288it [00:01, 193.62it/s]


In [17]:
test_datagen.fit(valid_images,augment=True)

In [21]:
nb_train_samples = len(train_images)
nb_validation_samples = len(valid_images)
epochs = 12
batch_size = 32
steps_per_epoch = nb_train_samples//batch_size
print(steps_per_epoch)
n_classes = 1

39


In [22]:
#Building a model
input_shape = (224, 224, 1)
model = build_model()

model.compile(loss=BinaryFocalLoss(gamma = 2), optimizer='adam', metrics=['acc', 'mse'])


51888128/51877672 [==============================] - 1s 0us/step


In [24]:
#call backs for early stopping incase of reduced learning rate, loss unimprovement
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, min_lr=0.0001)
callbacks_list = [early_stop, reduce_lr]

In [25]:
#train the module
model_history = model.fit(
    train_generator,
    epochs=epochs,
    workers=0,
    use_multiprocessing=False,  
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples //batch_size,
    callbacks=callbacks_list
    
)

Epoch 1/12
39/39 [==============================] - 70s 972ms/step - loss: 0.2666 - acc: 0.5798 - mse: 0.2494 - val_loss: 3.5818 - val_acc: 0.7778 - val_mse: 0.2222 - lr: 0.0010
Epoch 2/12
39/39 [==============================] - ETA: 0s - loss: 0.1666 - acc: 0.6661 - mse: 0.2218
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
39/39 [==============================] - 31s 804ms/step - loss: 0.1666 - acc: 0.6661 - mse: 0.2218 - val_loss: 5.3922 - val_acc: 0.5556 - val_mse: 0.3614 - lr: 0.0010
Epoch 3/12
39/39 [==============================] - 30s 758ms/step - loss: 0.1399 - acc: 0.7444 - mse: 0.2098 - val_loss: 1.0030 - val_acc: 0.2222 - val_mse: 0.4626 - lr: 1.0000e-04
Epoch 4/12
39/39 [==============================] - 30s 761ms/step - loss: 0.1325 - acc: 0.7500 - mse: 0.1967 - val_loss: 0.5856 - val_acc: 0.5556 - val_mse: 0.3530 - lr: 1.0000e-04
Epoch 5/12
39/39 [==============================] - 30s 757ms/step - loss: 0.1231 - acc: 0.7823 - mse: 0.18